In [1]:
import pandas as pd

df = pd.read_csv("../data_selected/car_hacking/DoS_dataset_aligned_train_bits.csv")

In [2]:
df.columns

Index(['Timestamp', 'CAN_ID_0', 'CAN_ID_1', 'CAN_ID_2', 'CAN_ID_3', 'CAN_ID_4',
       'CAN_ID_5', 'CAN_ID_6', 'CAN_ID_7', 'CAN_ID_8', 'CAN_ID_9', 'CAN_ID_10',
       'DLC', 'DATA_0', 'DATA_1', 'DATA_2', 'DATA_3', 'DATA_4', 'DATA_5',
       'DATA_6', 'DATA_7', 'DATA_8', 'DATA_9', 'DATA_10', 'DATA_11', 'DATA_12',
       'DATA_13', 'DATA_14', 'DATA_15', 'DATA_16', 'DATA_17', 'DATA_18',
       'DATA_19', 'DATA_20', 'DATA_21', 'DATA_22', 'DATA_23', 'DATA_24',
       'DATA_25', 'DATA_26', 'DATA_27', 'DATA_28', 'DATA_29', 'DATA_30',
       'DATA_31', 'DATA_32', 'DATA_33', 'DATA_34', 'DATA_35', 'DATA_36',
       'DATA_37', 'DATA_38', 'DATA_39', 'DATA_40', 'DATA_41', 'DATA_42',
       'DATA_43', 'DATA_44', 'DATA_45', 'DATA_46', 'DATA_47', 'DATA_48',
       'DATA_49', 'DATA_50', 'DATA_51', 'DATA_52', 'DATA_53', 'DATA_54',
       'DATA_55', 'DATA_56', 'DATA_57', 'DATA_58', 'DATA_59', 'DATA_60',
       'DATA_61', 'DATA_62', 'DATA_63', 'Label'],
      dtype='object')

In [3]:
gk = df.groupby([f'CAN_ID_{i}' for i in range(11)])

In [6]:
len(gk)

27

In [4]:
import pandas as pd

import math

def car_hacking_sessionize(df, num_per_session=10):
    df.fillna(0, inplace=True)

    grouped = df.groupby([f'CAN_ID_{i}' for i in range(11)])

    # Get max number of rows in a group
    print("Number of rows in the DataFrame: ", len(df))
    print("Number of groups: ", grouped.ngroups)
    print("Max number of rows in a group: ", grouped.size().max())

    # Global session ID counter
    global_session_id = 0

    # Iterate over each group
    for group_key, group in grouped:
        # print("Group key: ", group_key)
        # Calculate the number of sessions in this group
        total_rows = len(group)
        sessions_in_group = math.ceil(total_rows / num_per_session)  # Ceiling division

        # Assign session IDs within this group
        for session_number in range(sessions_in_group):
            # Determine the row indices for this session
            session_start = session_number * num_per_session
            session_end = session_start + num_per_session
            # Assign the global session ID to these rows
            df.loc[group.index[session_start:session_end], 'session_id'] = global_session_id
            # Increment the global session ID
            global_session_id += 1
    
    # Check if number of sessions is correct
    assert global_session_id == df['session_id'].nunique() == df.groupby(['session_id'] +  [f'CAN_ID_{i}' for i in range(11)]).ngroups
    print("Number of sessions: ", global_session_id)

    return df

final_df = car_hacking_sessionize(pd.read_csv("../data_selected/car_hacking/DoS_dataset_aligned_train_bits.csv"))

Number of rows in the DataFrame:  700000
Number of groups:  27
Max number of rows in a group:  111992
Number of sessions:  70015


In [5]:
for csv_name in ['DoS', 'Fuzzy', 'RPM', 'gear']:
    csv_filename = f"../data_selected/car_hacking/{csv_name}_dataset_aligned_train_bits"

    final_df = car_hacking_sessionize(pd.read_csv(f"{csv_filename}.csv"))
    final_df.to_csv(csv_filename + "_sessionized.csv", index=False)

    print(f"Finished {csv_name}")

Number of rows in the DataFrame:  700000
Number of groups:  27
Max number of rows in a group:  111992
Number of sessions:  70015
Finished DoS
Number of rows in the DataFrame:  700000
Number of groups:  2048
Max number of rows in a group:  33236
Number of sessions:  70916
Finished Fuzzy
Number of rows in the DataFrame:  700000
Number of groups:  26
Max number of rows in a group:  131665
Number of sessions:  70015
Finished RPM
Number of rows in the DataFrame:  700000
Number of groups:  26
Max number of rows in a group:  126904
Number of sessions:  70013
Finished gear
